In [ ]:
# !pip3 install dowhy --user 

In [ ]:
import dowhy

## Imports

In [ ]:
import dowhy
import numpy as np
import pandas as pd
from dowhy import CausalModel
import time

In [ ]:
import warnings
import logging

In [ ]:
matching_data = "../../matching_data/data_diab.p"
data = pd.read_pickle(matching_data)

In [ ]:
data.shape

In [ ]:
data.head(5)

## Some Processing

In [ ]:
A = data.loc[:,["AGE_AT_MIDMONTH", "MBR_GNDR", "ALLOW_AMT"]]
B = data.loc[:,"CC_ADHD":"CC_VISUAL_IMPAIR"]
X = A.merge(B, how='left', left_index=True, right_index=True)
T = data.TREATED
Y = data.OUTCOME

In [ ]:
data = pd.concat([X,Y,T], axis=1)
data = data.astype({"TREATED":'bool'}, copy=False)

In [ ]:
data.head()

## Run Log Reg for feature importance (Optional Block)
Here, we use Recursive Feature Elimination (RFE) technique provided by Scikit-Learn - https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html

In [ ]:
from sklearn import linear_model
from sklearn.feature_selection import RFE

In [ ]:
log_reg = linear_model.LogisticRegression(solver='lbfgs')
rfe = RFE(estimator = log_reg, n_features_to_select = 20, step=0.05)
rfe = rfe.fit(X,Y)

In [ ]:
import pickle
with open(r"RFE.pkl", "wb") as output_file:
    pickle.dump(rfe, output_file)
with open(r"RFE.pkl", "rb") as input_file:
    rfe = pickle.load(input_file)

In [ ]:
X.columns[rfe.support_]

In [ ]:
X = X[X.columns[rfe.support_]]

In [ ]:
data = pd.concat([X,Y,T], axis=1)
data = data.astype({"TREATED":'bool'}, copy=False)

## Apply Do Why - 

In [ ]:
common_causes = X.columns.tolist()
outcome = 'OUTCOME'

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
warnings.filterwarnings("ignore")

### 1. Propensty Score Matching

In [ ]:
%%time
#----------------------------
#step 1: Create Causal Model
#----------------------------
PSM_model=CausalModel(
        data = data,
        treatment='TREATED',
        outcome=outcome,
        common_causes=common_causes
        )
print("Step 1: Causal Model Created")
#------------------------------
#step 2: Identify the Estimand
#------------------------------
PSM_identified_estimand = PSM_model.identify_effect(proceed_when_unidentifiable=True)
print("Step 2: Estimand Identified")

#------------------------------
#step 3: Estimating the Effect
#------------------------------
PSM_estimate = PSM_model.estimate_effect(PSM_identified_estimand,
        method_name="backdoor.propensity_score_matching",
        target_units = "ate"
)
print("Step 3: Estimated the treatment effect")
print("Causal Estimate is " + str(PSM_estimate.value))

#------------------------------
#step 4: Refutation
#------------------------------
print("Step 4: Refutation")
#refute using common cause
#solution: Does the estimation method change its estimate after we add an independent random variable as a common cause to the dataset? (Hint: It should not)
refute_random_common_cause = PSM_model.refute_estimate(
                                        PSM_identified_estimand,
                                        PSM_estimate,
                                        method_name="random_common_cause"
                                    )
print("a. After random common cause refutation --")
print(refute_random_common_cause)

#refute using placebo treatment
#What happens to the estimated causal effect when we replace the true treatment variable with an independent random variable? (Hint: the effect should go to zero)
refute_placebo_treatment = PSM_model.refute_estimate(
                                    PSM_identified_estimand,
                                    PSM_estimate,
                                    method_name="placebo_treatment_refuter",
                                    placebo_type="permute"
                                )
print("b. After placebo treatment refutation --")
print(refute_placebo_treatment)

#refute using random subset
#Does the estimated effect change significantly when we replace the given dataset with a randomly selected subset? (Hint: It should not)
refute_random_subset = PSM_model.refute_estimate(
                                    PSM_identified_estimand,
                                    PSM_estimate,
                                    method_name="data_subset_refuter",
                                    subset_fraction=0.8
                            )
print("c. After random subset refutation --")
print(refute_random_subset)

### 2. Propensty Score Stratification

In [ ]:
%%time
#----------------------------
#step 1: Create Causal Model
#----------------------------
PSS_model=CausalModel(
        data = data,
        treatment='TREATED',
        outcome=outcome,
        common_causes=common_causes
        )
print("Step 1: Causal Model Created")

#------------------------------
#step 2: Identify the Estimand
#------------------------------
PSS_identified_estimand = PSS_model.identify_effect(proceed_when_unidentifiable=True)
print("Step 2: Estimand Identified")

#------------------------------
#step 3: Estimating the Effect
#------------------------------
PSS_estimate = PSS_model.estimate_effect(PSS_identified_estimand,
                                              method_name="backdoor.propensity_score_stratification",
                                              target_units="ate")
print("Step 3: Estimated the treatment effect")
print("Causal Estimate is " + str(PSS_estimate.value))

#------------------------------
#step 4: Refutation
#------------------------------
print("Step 4: Refutation")
#refute using common cause
#solution: Does the estimation method change its estimate after we add an independent random variable as a common cause to the dataset? (Hint: It should not)
refute_random_common_cause = PSS_model.refute_estimate(
                                        PSS_identified_estimand,
                                        PSS_estimate,
                                        method_name="random_common_cause"
                                    )
print("a. After random common cause refutation --")
print(refute_random_common_cause)

#refute using placebo treatment
#What happens to the estimated causal effect when we replace the true treatment variable with an independent random variable? (Hint: the effect should go to zero)
refute_placebo_treatment = PSS_model.refute_estimate(
                                    PSS_identified_estimand,
                                    PSS_estimate,
                                    method_name="placebo_treatment_refuter",
                                    placebo_type="permute"
                                )
print("b. After placebo treatment refutation --")
print(refute_placebo_treatment)

#refute using random subset
#Does the estimated effect change significantly when we replace the given dataset with a randomly selected subset? (Hint: It should not)
refute_random_subset = PSS_model.refute_estimate(
                                    PSS_identified_estimand,
                                    PSS_estimate,
                                    method_name="data_subset_refuter",
                                    subset_fraction=0.8
                            )
print("c. After random subset refutation --")
print(refute_random_subset)

### 3. Propensty Score Weighting

In [ ]:
%%time
#----------------------------
#step 1: Create Causal Model
#----------------------------
PSW_model=CausalModel(
        data = data,
        treatment='TREATED',
        outcome=outcome,
        common_causes=common_causes
        )
print("Step 1: Causal Model Created")

#------------------------------
#step 2: Identify the Estimand
#------------------------------
PSW_identified_estimand = PSW_model.identify_effect(proceed_when_unidentifiable=True)
print("Step 2: Estimand Identified")

#------------------------------
#step 3: Estimating the Effect
#------------------------------
PSW_estimate = PSW_model.estimate_effect(PSW_identified_estimand,
                                            method_name="backdoor.propensity_score_weighting",
                                            target_units = "ate",
                                            method_params={"weighting_scheme":"ips_weight"})
print("Step 3: Estimated the treatment effect")
print("Causal Estimate is " + str(PSW_estimate.value))

#------------------------------
#step 4: Refutation
#------------------------------
print("Step 4: Refutation")
#refute using common cause
#solution: Does the estimation method change its estimate after we add an independent random variable as a common cause to the dataset? (Hint: It should not)
refute_random_common_cause = PSW_model.refute_estimate(
                                        PSW_identified_estimand,
                                        PSW_estimate,
                                        method_name="random_common_cause"
                                    )
print("a. After random common cause refutation --")
print(refute_random_common_cause)

#refute using placebo treatment
#What happens to the estimated causal effect when we replace the true treatment variable with an independent random variable? (Hint: the effect should go to zero)
refute_placebo_treatment = PSW_model.refute_estimate(
                                    PSW_identified_estimand,
                                    PSW_estimate,
                                    method_name="placebo_treatment_refuter",
                                    placebo_type="permute"
                                )
print("b. After placebo treatment refutation --")
print(refute_placebo_treatment)

#refute using random subset
#Does the estimated effect change significantly when we replace the given dataset with a randomly selected subset? (Hint: It should not)
refute_random_subset = PSW_model.refute_estimate(
                                    PSW_identified_estimand,
                                    PSW_estimate,
                                    method_name="data_subset_refuter",
                                    subset_fraction=0.8
                           )
print(refute_random_subset)